# Tutorial 02 - Vanilla Learning

A vanilla learning process starts from scratch with the CommonRoad-RL environment `commonroad-v0` and a designated RL model, [PPO2](https://stable-baselines.readthedocs.io/en/master/modules/ppo2.html) for example. This tutorial explains the following operations:  
* how the RL environment and the RL model are configured before the learning begins
* how training and testing sessions are carried out differently
* how callback functions are created and attached to evaluate the learning
* how results are stored for later inspection or reuse

## 0. Preparation

Please make sure the training and testing data are prepared, otherwise see **Tutorial 01 - Data Preprocessing**. Also, check the followings:
* current path is at the project root `commonroad-rl`, i.e. one upper layer to the `tutorials` folder
* interactive python kernel is triggered from the correct environment

In [ ]:
# Check current path
%cd ..
%pwd

# Check interactive python kernel
import sys
sys.executable

## 1. Load RL environment and model settings

Before the learning begins, both the RL environment and the RL model are to be specified. This is done with the files of environment configurations and model hyperparameters respectively.  
To configure an environment, simply set the values in `commonroad_rl/config.yaml`. Please see the [commonroad_rl/gym_commonroad/README.md](https://gitlab.lrz.de/ss20-mpfav-rl/commonroad-rl/-/blob/development/commonroad_rl/gym_commonroad/README.md) file for a full description. To adjust the hyperparameters of a model, go to `commonroad_rl/hyperparams/{model}.yml`. Alternatively, the parameters can be read in and set with easy assignments.  
Furthermore, we save these settings for later use.

In [ ]:
import os
import yaml
import copy

# Read in environment configurations
env_configs = {}
with open("commonroad_rl/gym_commonroad/configs.yaml", "r") as config_file:
    env_configs = yaml.safe_load(config_file)["env_configs"]
    
# Change a configuration 
env_configs["reward_type"] = "dense_reward"

# Save settings for later use
log_path = "tutorials/logs/"
os.makedirs(log_path, exist_ok=True)

with open(os.path.join(log_path, "environment_configurations.yml"), "w") as config_file:
    yaml.dump(env_configs, config_file)

# Read in model hyperparameters
hyperparams = {}
with open("commonroad_rl/hyperparams/ppo2.yml", "r") as hyperparam_file:
    hyperparams = yaml.safe_load(hyperparam_file)["commonroad-v0"]
    
# Save settings for later use
with open(os.path.join(log_path, "model_hyperparameters.yml"), "w") as hyperparam_file:
    yaml.dump(hyperparams, hyperparam_file)
    
# Remove `normalize` as it will be handled explicitly later
if "normalize" in hyperparams:
    del hyperparams["normalize"]

## 2. Create a training environment
Now we are ready to create our customized RL environment `commonroad-v0` with the above configurations.  
Based on the API of [OpenAI gym](https://gym.openai.com/docs/), keyword arguments can be appended and passed over to the environment initializer. This comes in handy as we configure training/testing environments and specify data paths.  
Moreover, we utilize the [Monitor Wrapper](https://stable-baselines.readthedocs.io/en/master/common/monitor.html) and [Vectorized Environments](https://stable-baselines.readthedocs.io/en/master/guide/vec_envs.html) from OpenAI Stable Baselines to track and organize the learning process. 

In [ ]:
import gym
from stable_baselines.bench import Monitor
from stable_baselines.common.vec_env import DummyVecEnv, VecNormalize

import commonroad_rl.gym_commonroad

# Create a Gym-based RL environment with specified data paths and environment configurations
meta_scenario_path = "tutorials/data/highD/pickles/meta_scenario"
training_data_path = "tutorials/data/highD/pickles/problem_train"
training_env = gym.make("commonroad-v0", 
                        meta_scenario_path=meta_scenario_path,
                        train_reset_config_path= training_data_path,
                        **env_configs)

# Wrap the environment with a monitor to keep an record of the learning process
info_keywords=tuple(["is_collision", \
                     "is_time_out", \
                     "is_off_road", \
                     "is_friction_violation", \
                     "is_goal_reached"])
training_env = Monitor(training_env, log_path + "infos", info_keywords=info_keywords)

# Vectorize the environment with a callable argument
def make_training_env():
    return training_env
training_env = DummyVecEnv([make_training_env])

# Normalize observations and rewards
training_env = VecNormalize(training_env, norm_obs=True, norm_reward=True)

## 3. Create a testing environment
**Note**: Training environments are used the collect data to update the model. Testing environments are used to evaluate the performance of the model during training without updating the model.

Before starting the learning process, it is usual to prepare a testing environment to evaluate the training status during the process.  
For such, we simply append an additional key to the original environment configurations to create the testing environment. Then, we pass the testing environment to an evaluation callback, which constantly triggers several assessing episodes after a certain number, say 500 or 1000, of training steps.  
In addition, we create a customized [callback](https://stable-baselines.readthedocs.io/en/master/guide/callbacks.html) function to save the vectorized and normalized training environment wrapper whenever there is a new best model achieved during learning. This will be useful for later inspection and continual learning.

In [ ]:
from stable_baselines.common.callbacks import BaseCallback, EvalCallback

# Append the additional key
env_configs_test = copy.deepcopy(env_configs)
env_configs_test["test_env"] = True

# Create the testing environment
testing_data_path = "tutorials/data/highD/pickles/problem_test"
testing_env = gym.make("commonroad-v0", 
                        meta_scenario_path=meta_scenario_path,
                        test_reset_config_path= testing_data_path,
                        **env_configs_test)

# Wrap the environment with a monitor to keep an record of the testing episodes 
log_path_test = "tutorials/logs/test"
os.makedirs(log_path_test, exist_ok=True)

testing_env = Monitor(testing_env, log_path_test + "/infos", info_keywords=info_keywords)

# Vectorize the environment with a callable argument
def make_testing_env():
    return testing_env
testing_env = DummyVecEnv([make_testing_env])

# Normalize only observations during testing
testing_env = VecNormalize(testing_env, norm_obs=True, norm_reward=False)

# Define a customized callback function to save the vectorized and normalized environment wrapper
class SaveVecNormalizeCallback(BaseCallback):
    def __init__(self, save_path: str, verbose=1):
        super(SaveVecNormalizeCallback, self).__init__(verbose)
        self.save_path = save_path
        
    def _init_callback(self) -> None:
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)
    
    def _on_step(self) -> bool:
        save_path_name = os.path.join(self.save_path, "vecnormalize.pkl")
        self.model.get_vec_normalize_env().save(save_path_name)
        print("Saved vectorized and normalized environment to {}".format(save_path_name))
    
# Pass the testing environment and customized saving callback to an evaluation callback
# Note that the evaluation callback will triggers three evaluating episodes after every 500 training steps
save_vec_normalize_callback = SaveVecNormalizeCallback(save_path=log_path)
eval_callback = EvalCallback(testing_env, 
                             log_path=log_path, 
                             eval_freq=500, 
                             n_eval_episodes=3, 
                             callback_on_new_best=save_vec_normalize_callback)

## 4. Create a model and start the learning
Now we are ready to start a learning process. To such end, we conveniently instantiate a model provided by OpenAI Stable Baselines. For example, we create a PPO2 agent and learn for 5000 steps.

In [ ]:
from stable_baselines import PPO2

# Create the model together with its model hyperparameters and the training environment
model = PPO2(env=training_env, **hyperparams)

# Start the learning process with the evaluation callback
n_timesteps=5000
model.learn(n_timesteps, eval_callback)

As seen, there are evaluation messages being printed out after every 500 time steps. Additionally, the environment wrapper is saved whenever a best mean reward (of the three evaluating episodes) is obtained.  

Now we conclude this tutorial by saving the trained model so far.

In [ ]:
# Specify the filename and save the model
model.save("tutorials/logs/intermediate_model")

At this point, a zip file `intermediate_model.zip` and the settings, `environment_configurations.yml` and `model_hyperparameters.yml`, should be found under the designated path, alongside with  `evaluations.npz` created by the evaluation callback, `vecnormalize.pkl` by the saving callback, and `infos.monitor.csv` as well as `test/infos.monitor.csv` by the monitor wrappers. We shall take advantage of these files in the next tutorials.